In [1]:
from IPython.display import display

In [ ]:
from pdstools.explanations import Explanations

import datetime
import polars as pl

## Aggregate data exported from infinity

If this step has not already been done, you need to aggregate the explanation data exported from infinity. Unless otherwise specified, the resulting aggrgeated data will be stored in the `.tmp/aggregate/data` directory.

This step can be skipped if you have already aggregated the data.

In [ ]:
explanations = Explanations(
    data_folder='../../data/explanations/',
    model_name='AdaptiveBoostCT',
    to_date=datetime.date(2025,3,28)
)
explanations.aggregates.generate()

In [ ]:
explanations.reports.generate()
explanations.explorer.display()

## Simple Data Exploration
For quick overview of the data, you can use the ExplanationsExplorer class. It provides a simple interface to select a context and then plot the corrsponding predictor contribution plots.

In [4]:
explorer = ExplanationsExplorer()

### Selecting a context
Select the desired context from the list available on the right.
* Selecting 'Any' means the plots will display and aggegation across all contexts.
* Selecting a specific context will result in plots which aggregate the data for that context only.

If you have a very large list of possible contexts, you can filter the list on the right by selecting specific context keys from the comboboxes on the left.

In [5]:
explorer.display_context_selector()

GridBox(children=(HTML(value='<h3>Select Context Filters</>', layout=Layout(width='auto')), HTML(value='<h3>Co…

## Plotting the explanation data

The method shown below can be used to plot the explanation data for context selected above.
* The first plot shows the top 10 predictors sorted by their average contributions to predictions, the number can be changed by passing a different value to the `top_n` parameter.
* The subsequent plots show the avergae prediction contributions for the different values of top predictors.
    * Numeric predictor values are binned into deciles.
    * Categorical predictors are sorted and limited to the top 10 highest contributing values.

Note that the number of top predictors and top values can be changed by passing different values for the `top_n` and `top_k` parameters, respectively.

Additionally, if you are interested seeing the least contribution predictors, the `descending` parameter can be set to `False`. This will plot the least contributing predictors instead of the most contributing ones.

In [6]:
plots = explorer.plot_contributions(top_n = 5, top_k = 5)

No context selected, plotting overall contributions.


## Advaced Data Exploration
For more advanced data explortation you can directly use the ExplanationsDataLoader and ExplanationsDataLoader classes. These classes provide more flexibility in how the data is loaded and processed. Allowing you to inspect the data before plotting.

In [7]:
loader = ExplanationsDataLoader(data_location='.tmp/aggregated_data')

In [8]:
top_predictors = loader.get_top_n_predictor_contribution_overall(top_n = 5, remaining=False)
top_predictors

partition,predictor_name,predictor_type,contribution,contribution_abs,contribution_weighted,contribution_weighted_abs,frequency,contribution_min,contribution_max
str,str,str,f64,f64,f64,f64,i64,f64,f64
"""whole_model""","""pyName""","""SYMBOLIC""",-0.021191,0.021255,-0.000096,0.000097,50000,-0.044283,0.029203
"""whole_model""","""Age""","""NUMERIC""",-0.011306,0.011738,-0.000092,0.000095,50000,-0.034704,0.023485
"""whole_model""","""Occupation""","""SYMBOLIC""",-0.008904,0.010193,-0.000017,0.000019,50000,-0.032185,0.06411
"""whole_model""","""CustomerName""","""SYMBOLIC""",-0.005215,0.005814,-4.2977e-7,5.4566e-7,50000,-0.024571,0.029192
"""whole_model""","""NumX""","""NUMERIC""",-0.003837,0.005153,-0.000031,0.000042,50000,-0.025888,0.029391


We can get the top predictors and inspect their most influential values

In [9]:
predictors = top_predictors.select(pl.col('predictor_name')).unique().to_series().to_list()
loader.get_top_k_predictor_value_contribution_overall(predictors=predictors, top_k = 5, remaining=False)

partition,predictor_name,predictor_type,bin_order,bin_contents,contribution,contribution_abs,contribution_weighted,contribution_weighted_abs,frequency,contribution_min,contribution_max,sort_column,sort_value
str,str,str,i64,str,f64,f64,f64,f64,i64,f64,f64,str,f64
"""whole_model""","""Age""","""NUMERIC""",0,"""MISSING""",-0.013647,0.015131,-0.000503,0.000557,1842,-0.034704,0.023485,"""bin_order""",0.0
"""whole_model""","""Age""","""NUMERIC""",4,"""[38.000:43.000]""",-0.012799,0.013064,-0.001233,0.001258,4816,-0.026804,0.014212,"""bin_order""",4.0
"""whole_model""","""Age""","""NUMERIC""",5,"""[43.000:48.000]""",-0.013128,0.013146,-0.001265,0.001266,4816,-0.027359,0.006043,"""bin_order""",5.0
"""whole_model""","""Age""","""NUMERIC""",6,"""[48.000:53.000]""",-0.012951,0.012955,-0.001247,0.001248,4816,-0.024634,0.005957,"""bin_order""",6.0
"""whole_model""","""Age""","""NUMERIC""",7,"""[53.000:58.000]""",-0.012529,0.012567,-0.001207,0.00121,4816,-0.025815,0.009732,"""bin_order""",7.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""whole_model""","""pyName""","""SYMBOLIC""",5,"""P19""",-0.02322,0.02324,-0.001171,0.001172,2522,-0.035388,0.008371,"""contribution""",0.02322
"""whole_model""","""pyName""","""SYMBOLIC""",4,"""P15""",-0.023321,0.023321,-0.001183,0.001183,2536,-0.034834,-0.003991,"""contribution""",0.023321
"""whole_model""","""pyName""","""SYMBOLIC""",3,"""P3""",-0.023462,0.023462,-0.001197,0.001197,2550,-0.034775,-0.006639,"""contribution""",0.023462


Let's repeat the same again, but this time we will inspect a specific context, instead of the entire model.

In [10]:
import random
context_info = random.choice(loader.get_unique_contexts_list())
print('Selected random context: \n')
for key, value in context_info.items():
    print(f'{key}: {value}')
top_predictors_for_selected_context = loader.get_top_n_predictor_contribution_by_context(context=context_info, top_n=5, remaining=False)
top_predictors_for_selected_context


Selected random context: 

pyChannel: PegaBatch
pyDirection: E2E Test
pyGroup: E2E Test
pyIssue: Batch
pyName: P1


partition,predictor_name,predictor_type,contribution,contribution_abs,contribution_weighted,contribution_weighted_abs,frequency,contribution_min,contribution_max
str,str,str,f64,f64,f64,f64,i64,f64,f64
"""{""partition"":{""pyChannel"":""Peg…","""pyName""","""SYMBOLIC""",-0.025354,0.025354,-0.002305,0.002305,2577,-0.044283,-0.007185
"""{""partition"":{""pyChannel"":""Peg…","""Age""","""NUMERIC""",-0.011494,0.012012,-0.000096,0.000099,2577,-0.026804,0.014561
"""{""partition"":{""pyChannel"":""Peg…","""Occupation""","""SYMBOLIC""",-0.009801,0.010404,-0.000018,0.000019,2577,-0.023662,0.030971
"""{""partition"":{""pyChannel"":""Peg…","""CustomerName""","""SYMBOLIC""",-0.004977,0.005567,-5.0844e-7,6.4745e-7,2577,-0.019922,0.022087
"""{""partition"":{""pyChannel"":""Peg…","""NumX""","""NUMERIC""",-0.003691,0.004775,-0.00003,0.000039,2577,-0.013727,0.015084


In [11]:
predictors_for_selected_context = top_predictors_for_selected_context.select(pl.col('predictor_name')).unique().to_series().to_list()
loader.get_top_k_predictor_value_contribution_by_context(predictors=predictors_for_selected_context, top_k=5, context=context_info, remaining=False)

partition,predictor_name,predictor_type,bin_order,bin_contents,contribution,contribution_abs,contribution_weighted,contribution_weighted_abs,frequency,contribution_min,contribution_max,sort_column,sort_value
str,str,str,i64,str,f64,f64,f64,f64,i64,f64,f64,str,f64
"""{""partition"":{""pyChannel"":""Peg…","""Age""","""NUMERIC""",2,"""[25.000:33.000]""",-0.013923,0.014139,-0.001334,0.001355,247,-0.026038,0.005331,"""bin_order""",2.0
"""{""partition"":{""pyChannel"":""Peg…","""Age""","""NUMERIC""",3,"""[33.000:38.000]""",-0.014624,0.014624,-0.001402,0.001402,247,-0.025613,-0.001506,"""bin_order""",3.0
"""{""partition"":{""pyChannel"":""Peg…","""Age""","""NUMERIC""",4,"""[38.000:43.000]""",-0.016419,0.01649,-0.001574,0.00158,247,-0.026804,0.001462,"""bin_order""",4.0
"""{""partition"":{""pyChannel"":""Peg…","""Age""","""NUMERIC""",5,"""[43.000:48.000]""",-0.0148,0.0148,-0.001419,0.001419,247,-0.025701,-0.005414,"""bin_order""",5.0
"""{""partition"":{""pyChannel"":""Peg…","""Age""","""NUMERIC""",6,"""[48.000:52.000]""",-0.013537,0.013537,-0.001298,0.001298,247,-0.019743,-0.005095,"""bin_order""",6.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""{""partition"":{""pyChannel"":""Peg…","""Occupation""","""SYMBOLIC""",41,"""Therapist, speech and language""",-0.014844,0.014844,-0.000242,0.000242,42,-0.023662,-0.002825,"""contribution""",0.014844
"""{""partition"":{""pyChannel"":""Peg…","""Occupation""","""SYMBOLIC""",27,"""Interior and spatial designer""",-0.014858,0.014858,-0.000277,0.000277,48,-0.02143,-0.002198,"""contribution""",0.014858
"""{""partition"":{""pyChannel"":""Peg…","""Occupation""","""SYMBOLIC""",38,"""Psychotherapist, child""",-0.015239,0.015239,-0.000254,0.000254,43,-0.02263,-0.007052,"""contribution""",0.015239
